# 抛物方程有限差分求解编程示例

* 利用 SymPy 中 `diff`、 `sympify` 和 `lambdify` 函数编写更通用的 PDE 方程模型
* 演示数值计算编程的最佳实践原则和流程
* 引入 PDE 模型的基类

## 一、 一维有限差分

### 算例一

考虑如下抛物方程问题：
$$\begin{cases}
&\dfrac{\partial u}{\partial t}-\dfrac{\partial^2u}{\partial x^2}=f(x,t),\\
& u(0,t)=0,\quad u(1,t)=0,\\
& u(x,0)=u_0(x).
\end{cases}$$
其中$k=1$。给定一个具体的真解
$$u(x,t) = \sin(4\pi x)e^{-10t}.$$

首先利用 SymPy 编写更通用的 PDE 数据模型类，并强调测试程序正确性的重要性。

In [3]:
import numpy as np
from sympy import *

class ParabolicPDEData1D: 
    def __init__(self, u:str, x: str='x', t: str='t', k=1.0, D=[0, 1], T=[0, 1]):
        """
        @brief 模型初始化函数
        @param[in] D 模型空间定义域
        @param[in] T 模型时间定义域
        """
        self.u = lambdify([x, t], simplify(u))
        self.f = lambdify([x, t], diff(u, t,1) - k*diff(u, x, 2))
        self.dudx = lambdify([x, t], diff(u, x, 1))
        self._domain = D 
        self._duration = T 

    def domain(self):
        """
        @brief 空间区间
        """
        return self._domain

    def duration(self):
        """
        @brief 时间区间
        """
        return self._duration 
        
    def solution(self, p, t):
        """
        @brief 真解函数

        @param[in] p numpy.ndarray, 空间点
        @param[in] t float, 时间点 

        @return 真解函数值
        """
        return self.u(p, t) 

    def init_solution(self, p):
        """
        @brief 真解函数

        @param[in] p numpy.ndarray, 空间点
        @param[in] t float, 时间点 

        @return 真解函数值
        """
        return self.u(p, 0.0)
        
    def source(self, p, t):
        """
        @brief 方程右端项 

        @param[in] p numpy.ndarray, 空间点
        @param[in] t float, 时间点 

        @return 方程右端函数值
        """
        return self.f(p, t)
    
    def gradient(self, p, t):
        """
        @brief 真解导数 

        @param[in] p numpy.ndarray, 空间点
        @param[in] t float, 时间点 

        @return 真解导函数值
        """
        return self.dudx(p, t)

    def dirichlet(self, p, t):
        """
        @brief Dirichlet 边界条件

        @param[in] p numpy.ndarray, 空间点
        @param[in] t float, 时间点 
        """
        return self.solution(p, t)

pde = ParabolicPDEData1D('sin(4*pi*x)*exp(-10*t)')

最后，我们演示如何基于 FEALPy 编写整个有限差分求解一维椭圆方程的程序，并进行误差分析。

In [ ]:
from fealpy.mesh import UniformMesh1d

pde = ParabolicPDEData('sin(4*pi*x)*exp(-10*t)')

domain = pde.domain()
nx = 10
hx = (domain[1] - domain[0])/nx
mesh = UniformMesh1d([0, nx], h=hx, origin=domain[0])

duration = pde.duration()
nt =400
tau = (duration[1] - duration[0])/nt

uh0 = mesh.interpolate(pde.init_solution, intertype='node')


In [ ]:

def advance_forward(n: np.int_):
    """
    @brief 时间步进格式为向前欧拉方法

    @param[in] n int, 表示第 n 个时间步（当前时间步） 
    """
    t = duration[0] + n*tau
    if n == 0:
        return uh0, t
    else:
        A = mesh.parabolic_operator_forward(tau)
        source= lambda p: pde.source(p, t + tau)
        f = mesh.interpolate(source, intertype='node')
        uh0[:] = A@uh0 + tau*f
        gD = lambda p: pde.dirichlet(p, t + tau)
        mesh.update_dirichlet_bc(gD, uh0)
        
        solution = lambda p: pde.solution(p, t + tau)
        e = mesh.error(solution, uh0, errortype='max')
        print(f"the max error is {e}")
        return uh0, t


fig, axes = plt.subplots()
box = [0, 1, -1.5, 1.5] # 图像显示的范围 0 <= x <= 1, -1.5 <= y <= 1.5

mesh.show_animation(fig, axes, box, advance_forward, fname='advance_forward.mp4', 
                    frames=nt+1, lw=2, interval=50, linestyle='--', color='red')
plt.show()

# 二维有限差分
给定如下抛物方程问题：
$$\begin{cases}
&\dfrac{\partial u}{\partial t} - k\left(\dfrac{\partial^2 u}{\partial x^2} + \dfrac{\partial^2 u}{\partial y^2}\right) = -20e^{-20t}\sin(4\pi x)\sin(4\pi y) + 32\pi^2e^{-20t}\sin(4\pi x)\sin(4\pi y),\\
& u(x, 0, t) = 0,\quad u(x, 1, t) = 0,\\
& u(0, y, t) = 0,\quad u(1, y, t) = 0,\\
& u(x, y, 0) = u_0(x, y).
\end{cases}$$
其中$k=1$。给定一个具体的真解：
$$u(x, y, t) = \sin(4\pi x)\sin(4\pi y)e^{-20t}.$$